In [23]:
import os
from dotenv import load_dotenv 
from langchain_groq import ChatGroq
from langchain_core.messages import  HumanMessage, SystemMessage, AnyMessage
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from typing import Annotated
from langgraph.graph.message import add_messages


os.environ["LANGSMITH_PROJECT"] = "AgenticAIworkspace"
load_dotenv()
groq_key = os.getenv("GROQ_API_KEY")  

In [24]:
# Initialize the LLM
llm = ChatGroq(api_key=groq_key, model="llama-3.1-8b-instant")

In [25]:
# Define a simple tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

# This will be a tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b

def divide(a: int, b: int) -> float:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a / b


In [26]:
tools = [add, multiply, divide]
llm_with_tools = llm.bind_tools(tools)

In [27]:
# Define State
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

# Node
def assistant(state: State):
    return {"messages": [llm_with_tools.invoke([sys_message] + state["messages"])]}

sys_message = SystemMessage(content="You are a helpful assistant tasked with performing arithmetic on a set of inputs.")


In [28]:
# Build graph
builder = StateGraph(State)
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", tools_condition)
builder.add_edge("tools", "assistant") # <- back to the brain and make one more decition
builder.add_edge("assistant", END)  # Terminate graph

graph = builder.compile()


In [29]:

# Input
input_messages = [HumanMessage(content="Add 3 and 4. Multiply the output by 2. Divide the output by 5 and what is machin learning (ML) and LLM?")]

# Invoke graph
result = graph.invoke({"messages": input_messages})

# Print messages
for msg in result["messages"]:
    print(msg)

content='Add 3 and 4. Multiply the output by 2. Divide the output by 5 and what is machin learning (ML) and LLM?' additional_kwargs={} response_metadata={} id='9932901e-965b-479d-b5aa-cf31c4966f02'
content='' additional_kwargs={'tool_calls': [{'id': '34wtky43k', 'function': {'arguments': '{"a":3,"b":4}', 'name': 'add'}, 'type': 'function'}, {'id': '97nqd45j8', 'function': {'arguments': '{"a":7,"b":2}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'z2r1x7s3w', 'function': {'arguments': '{"a":14,"b":5}', 'name': 'divide'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 207, 'prompt_tokens': 398, 'total_tokens': 605, 'completion_time': 0.420465436, 'prompt_time': 0.028885059, 'queue_time': 0.063912444, 'total_time': 0.449350495}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_7b3cfae3af', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--787d5e9f-4fde-42c8-b606-ec79e91c5784-0' tool_calls=[{'name': 'ad

In [31]:

# Input
input_messages = [HumanMessage(content="What is javascript?")]

# Invoke graph
result = graph.invoke({"messages": input_messages})

# Print messages
for msg in result["messages"]:
    print(msg)

content='What is javascript?' additional_kwargs={} response_metadata={} id='348d8f47-2596-4bf7-b051-7d4cea226806'
content='<brave_search>' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 369, 'total_tokens': 374, 'completion_time': 0.012374239, 'prompt_time': 0.020443733, 'queue_time': 0.051289442, 'total_time': 0.032817972}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_e32974efee', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--f184a349-d99e-47c1-8747-5b289f49b369-0' usage_metadata={'input_tokens': 369, 'output_tokens': 5, 'total_tokens': 374}


 #### Memory Sever

In [ ]:
# Build graph
builder_builder = StateGraph(State)
builder_builder.add_node("assistant", assistant)
builder_builder.add_node("tools", ToolNode(tools))

builder_builder.add_edge(START, "assistant")
builder_builder.add_conditional_edges("assistant", tools_condition)
builder_builder.add_edge("tools", "assistant") # <- back to the brain and make one more decition
builder_builder.add_edge("assistant", END)  # Terminate graph



In [ ]:

from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

graph = builder_builder.compile(checkpointer=memory)


In [38]:
# specify the thread
config = {"configurable": {"thread_id": "1"}}
input_messages = [HumanMessage(content="add 3 and 4")]
result = graph.invoke({"messages": input_messages}, config=config)

for m in result["messages"]:
    print(m)

content='add 3 and 4' additional_kwargs={} response_metadata={} id='7354dd30-03ee-4a93-8d03-291cb0ca8b30'
content='' additional_kwargs={'tool_calls': [{'id': 'jz8d2nrd7', 'function': {'arguments': '{"a":3,"b":4}', 'name': 'add'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 371, 'total_tokens': 389, 'completion_time': 0.028099938, 'prompt_time': 0.02041292, 'queue_time': 0.05267431, 'total_time': 0.048512858}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_33e8adf159', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--fe3c0ce3-292c-4e6e-aa35-ac01db3b5cfb-0' tool_calls=[{'name': 'add', 'args': {'a': 3, 'b': 4}, 'id': 'jz8d2nrd7', 'type': 'tool_call'}] usage_metadata={'input_tokens': 371, 'output_tokens': 18, 'total_tokens': 389}
content='7' name='add' id='1ab40591-54f2-40e0-bbd6-0872128b2f8d' tool_call_id='jz8d2nrd7'
content='' additional_kwargs={'tool_calls': [{'id': '6gkky3k2q',